# Создание и заполнение данных БД Postgre

In [4]:
%pip install python-dotenv psycopg2-binary

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
import json
import psycopg2
from psycopg2.extras import DictCursor
from dotenv import load_dotenv


# Получение секретов

In [6]:
# получаем текущую директорию ноутбука 
current_dir = os.getcwd()

# переходим на один уровень вверх
project_root = os.path.dirname(current_dir)

# формируем путь к файлу .env в папке Task1, там у нас лежит файл .env с настройками подключения к БД
dotenv_path = os.path.join(project_root, 'task_1_Docker', '.env')

# загружаем переменные окружения из указанного файла
load_dotenv(dotenv_path)

# получим доступ к переменным окружения
user = os.getenv("USER")
password = os.getenv("PASSWORD")
db_name = os.getenv("DB")
db_port = os.getenv("DB_PORT") 
secret_hash = os.getenv("SECRET_HASH") 

print(f"Загруженные данные: USER={user}, DB={db_name}, DB_PORT={db_port}")

Загруженные данные: USER=katy199812, DB=user_logs_db, DB_PORT=5433


# Подключение к базе данных PostgreSQL

In [9]:
conn = None
try:
    conn = psycopg2.connect(
        host="localhost", # если Docker контейнер запущен локально, а ноутбук вне Docker.
                          # НО! если ноутбук также в Docker и в одной сети с БД,
                          # то нужно использовать имя сервиса Docker (например, 'db' или 'postgres_db').
        database=db_name,
        user=user,
        password=password,
        port=db_port
    )
    cursor = conn.cursor()

    print("Успешное подключение к базе данных!")
    
except Exception as e:
    print(f"Ошибка при подключении к базе данных: {e}")

Успешное подключение к базе данных!


In [10]:
# пример запроса
cursor.execute("SELECT version();")
db_version = cursor.fetchone()
print(f"Версия PostgreSQL: {db_version}")

Версия PostgreSQL: ('PostgreSQL 13.22 (Debian 13.22-1.pgdg13+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 14.2.0-19) 14.2.0, 64-bit',)


In [11]:
# получить список таблиц:
cursor.execute("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
""")
tables = cursor.fetchall()
print("\nТаблицы в базе данных:")
for table in tables:
    print(f"- {table[0]}")


Таблицы в базе данных:
- user_logs


In [ ]:
# закрытие соединения с БД - После завершения работы с БД не забываем закрывать соединение!
cursor.close()
conn.close()

Вам предоставлена БД с логами (действиями) студентов на образовательном портале за весенний семестр (агрегация по каждой неделе) по отдельному электронному курсу - таблица user_logs (примечание. создана в предыдущих л.р.).
- сourseid — уникальный идентификатор курса, дисциплины;
- userid — уникальный идентификатор студента (не используется в обучении);
- num_week — номер недели в году;
- s_all — количество всех событий на текущий момент;
- s_all_avg — среднее количество всех событий в неделю;
- s_course_viewed — количество просмотров курса;
- s_course_viewed_avg — среднее количество просмотров курса в неделю;
- s_q_attempt_viewed — количество просмотров теста;
- s_q_attempt_viewed_avg — среднее количество просмотров теста в неделю;
- s_a_course_module_viewed — количество просмотров модуля в курсе;
- s_a_course_module_viewed_avg — среднее количество просмотров модуля в курсе в неделю;
- s_a_submission_status_viewed — количество отправленных заданий на проверку;
- s_a_submission_status_viewed_avg — среднее количество ответов;
- namer_level — оценка за дисциплину;
- depart — номер кафедры;
- name_osno — основа обучения (имеет два значения: бюджет или контракт);
- name_formopril — форма обучения;
- leveled — уровень образования (имеет два значения: бакалавриат, магистратура);
- num_sem — номер семестра;
- kurs — номер курса учебной группы.

Также в таблице  departments хранятся названия кафедр, таблица связана с логами по полю depart:
id - код кафедры;
name - сокращенное название кафедры. 

## Задание 1 (если до этого еще этот шаг не был выполнен):

Измените данные вещественного типа, сейчас целая и дробная часть разделены запятой, замените ее на точку. 

Выведите первые 10 записей, чтобы проверить результат предобработки. 

## Задание 2: 

Выведите количество кафедр, за которыми закреплены курсы на портале.





##  Задание 3:

Выведите сколько у каждой кафедры закреплено электронных курсов на портале. 
Требуется выводит сокращенное название кафедры и количество курсов. 
У какой кафедры больше всего курсов на портале?

## Задание 4:

Ответьте на вопрос: существуют ли курсы, за которыми закреплено несколько кафедр? Если такие курсы есть, то выведите их количество.
Также выведите названия кафедр, которые совместно преподают один и тот же курс.




## Задание 5:

Выведите количество студентов, которые получили 2, 3, 4, 5.

Пример вывода:

| namer_level |	count |
|-----|------|
|2 |	4 |
|3 |	3435 |
|4 | 	4676765|
|5 | 232 |


## Задание 6:

Выведите студента, который больше всех работает на портале (у него максимальное количество логов за вест период обучения).

## Задание 7:

Выведите по каждой недели среднее количество всех событий на портале.

## Задание 8: 

Выведите название кафедры, у которой больше всего отличников.

Отдельно выведите название кафедры, у которой больше всего двоечников. 

## Задание 9:
Провести анализ пиковой активности студентов перед экзаменом (с использованием (Common Table Expression — CTE), оператор with).

Вывести, на какой неделе семестра студенты проявляли наибольшую активность в курсе в целом, и как эта активность распределяется между студентами-бюджетниками и контрактниками.

Пример вывода :

| name_osno | week_number	| avg_s_all	| avg_s_course_viewed |	avg_s_q_attempt_viewed |
|-----|------|------|------|------|
| бюджет |	14	| 125.45 |	45.67 |	32.12 |
|контракт |	14	| 98.76 |	38.90 |	25.43 |